In [23]:
import advent
import numpy as np
from advent.maze import solve_maze, solve_maze_no_tqdm
data = advent.get_char_grid(18)

def unravel(tuple_ix):
    return tuple_ix[0][0], tuple_ix[1][0]

In [54]:
# I'm going to try a two-stage approach: first use dijkstra to
# find the shortest paths between all pairs of keys, then use
# dijkstra to find the shortest path that visits all keys.

key_names = '@abcdefghijklmnopqrstuvwxyz'
door_names = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
keys, doors = {}, {}
for key in key_names:
    keys[key] = unravel(np.where(data == key))
for door in door_names:
    doors[unravel(np.where(data == door))] = door


In [61]:
def adjacent(coord):
    x, y = coord
    for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)):
        new_coord = x + dx, y + dy
        if data[new_coord] != '#':
            yield new_coord, 1

def manhattan(coord1, coord2):
    return abs(coord1[0] - coord2[0]) + abs(coord1[1] - coord2[1])

def get_h(target_key):
    return lambda coord: manhattan(coord, keys[target_key])

def get_blockers(path, coord):
    # coord is the ending position of the path
    blockers = []
    while coord is not None:
        coord = path[coord]
        if coord in doors:
            blockers.append(doors[coord].lower())
    return blockers

all_paths = {}

from tqdm import tqdm
for start_key in tqdm(keys):
    for target_key in keys:
        result = solve_maze_no_tqdm(
            keys[start_key],
            lambda coord: coord == keys[target_key],
            adjacent,
            h=get_h(target_key)
        )
        all_paths[(start_key, target_key)] = result[0], get_blockers(result[1], keys[target_key])


100%|██████████| 27/27 [00:06<00:00,  4.13it/s]


In [ ]:
from typing import NamedTuple
class Node(NamedTuple):
    current_key: str
    keys: frozenset[str] # must include current_key

    def adjacent(self):
        for target_key in keys:
            if target_key in self.keys: continue
            path, blockers = all_paths[(self.current_key, target_key)]
            if all(blocker in self.keys for blocker in blockers):
                yield Node(target_key, self.keys | {target_key}), path

def is_target(node):
    return len(node.keys) == len(keys)

start = Node('@', frozenset(['@']))

result = solve_maze(
    start,
    is_target,
    lambda node: node.adjacent()
)

# Part 2

contains some repetition from part 1 because both steps of dijkstra need to be redone slightly differently

performance-wise it should be about the same since there are the same number of possibilies of orders to collect keys (26 factorial), we just need to track 4 robot positions so Node will become slightly more complex

also not all paths are possible so 'all_paths' need to be recomputed

In [82]:
# part2
data = data.copy()
data[39:42, 39:42] = np.array([['@', '#', '$'], ['#', '#', '#'], ['%', '#', '^']])
data[39:42, 39:42]

key_names = '@$%^abcdefghijklmnopqrstuvwxyz'
door_names = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
keys, doors = {}, {}
for key in key_names:
    keys[key] = unravel(np.where(data == key))
for door in door_names:
    doors[unravel(np.where(data == door))] = door

In [85]:
all_paths = {}

from tqdm import tqdm
for start_key in tqdm(keys):
    for target_key in keys:
        result = solve_maze_no_tqdm(
            keys[start_key],
            lambda coord: coord == keys[target_key],
            adjacent,
            h=get_h(target_key)
        )
        if result[2]:
            all_paths[(start_key, target_key)] = result[0], get_blockers(result[1], keys[target_key])
        else:
            all_paths[(start_key, target_key)] = None, None

100%|██████████| 30/30 [00:03<00:00,  8.93it/s]


In [ ]:
from typing import NamedTuple
class Node(NamedTuple):
    current_key: tuple[str, str, str, str]
    keys: frozenset[str] # must include current_key

    def adjacent(self):
        #print(self.current_key, self.keys)
        for target_key in keys:
            if target_key in self.keys: continue
            for i, curr in enumerate(self.current_key):
                path, blockers = all_paths[(curr, target_key)]
                if not path: continue # this robot can't reach target_key
                if all(blocker in self.keys for blocker in blockers):
                    new_current_key = list(self.current_key)
                    new_current_key[i] = target_key
                    yield Node(
                        tuple(new_current_key),
                        keys=self.keys | {target_key}
                    ), path

def is_target(node):
    return len(node.keys) == len(keys)

start = Node(('@', '$', '%', '^'), frozenset(['@', '$', '%', '^']))

result = solve_maze(
    start,
    is_target,
    lambda node: node.adjacent()
)